In [2]:

!pip install -qU transformers datasets


In [ ]:
import numpy as np
import pandas as pd
from IPython import display
from datasets import load_dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments


In [ ]:

data = load_dataset("pn_summary")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

display.display(display.HTML("""

""".strip()))


train = data["train"]
indices = np.random.randint(0, len(train), 10).tolist()
df = pd.DataFrame(train[indices])
df = df[["id", "title", "article", "summary", "category", "categories", "network", "link"]]
df["article"] = df["article"].apply(lambda t: t.replace('[n]', ' ')[:512] + ' [...]')
df["category"] = df["category"].apply(lambda t: train.features["category"].int2str(t))
df["network"] = df["network"].apply(lambda t: train.features["network"].int2str(t))

setup = {
    'border': 2,
    'show_dimensions': True,
    'justify': 'right',
    'classes': 'xxx'
}
display.display(display.HTML(df.to_html(**setup)))

,id,title,article,summary,category,categories,network,link
0,4597017eeddcbbbdd4d1015c7e24b5c467b55e50,آمار کرونا کرمانشاه ۲۲ آبان؛ ۸ فوتی و ۴۹۲ ابتلای جدید,مهدی محمدی در گفت‌وگو با خبرنگار ایمنا، در خصوص آخرین آمار کلی کرونا در استان کرمانشاه در ۲۴ ساعت منتهی به ظهر امروز پنجشنبه بیست و دوم آبان ماه- اظهار کرد: در حال حاضر یک هزار و ۲۳۹ نفر مبتلای قطعی به کرونا در بیمارستان‌های استان کرمانشاه بستری هستند. وی با بیان اینکه طی ۲۴ ساعت گذشته ۴۹۲ بیمار مبتلای قطعی کرونا در کرمانشاه شناسایی شده‌است، ادامه داد: از این تعداد ۳۶۸ مورد جز بیماران تست مثبت سرپایی و ۹۴ مورد جز بیماران تست مثبت بستری هستند. رئیس کمیته اطلاع رسانی ستاد مدیریت کرونا دانشگاه علوم پزشکی [...],رئیس کمیته اطلاع‌رسانی ستاد مدیریت کرونا دانشگاه علوم پزشکی کرمانشاه آخرین آمار مبتلایان، بهبود یافتگان و فوتی‌های مبتلا به کرونا بیست و دوم آبان ماه را تشریح کرد.,Society,جامعه,Imna,https://www.imna.ir/news/455044/%D8%A2%D9%85%D8%A7%D8%B1-%DA%A9%D8%B1%D9%88%D9%86%D8%A7-%DA%A9%D8%B1%D9%85%D8%A7%D9%86%D8%B4%D8%A7%D9%87-%DB%B2%DB%B2-%D8%A2%D8%A8%D8%A7%D9%86-%DB%B8-%D9%81%D9%88%D8%AA%DB%8C-%D9%88-%DB%B4%DB%B9%DB%B2-%D8%A7%D8%A8%D8%AA%D9%84%D8%A7%DB%8C-%D8%AC%D8%AF%DB%8C%D8%AF
1,72cfd5e2355260d09f5bee3ad90dc309965982fb,مجموعه شعر گیو نصیری منتشر شد,به گزارش روز شنبه خبرنگار فرهنگی ایرنا، عالیجناب آتش افروز دومین مجموعه اشعار گیو نصیری است که پیش از آن کتاب سیراب از سراب را منتشر کرده بود. این مجموعه شعر در قالب‌های شعری آزاد و سپید با نگاهی مدرن پرداخته شده‌اند و شاعر به دنبال دستیابی و معرفی سبکی شخصی و بیانگری منحصر به خود است. به همین سبب گاهی در پاره‌ای از شعرها مخاطب با پیچیدگی‌های فرمی مواجهه می‌شود. برای مثال در یکی از شعرها با نام جاذبه‌های چپ ساعت که شعری روایی ست، جریان روایت از انتها به آغاز می‌رود یا در شعری دیگر به نام پیوستار دیدن و [...],مجموعه شعر عالیجناب آتش افروز سروده گیو نصیری به تازگی توسط انتشارات نگاه معاصر منتشر و راهی باز شده است.,Art-Culture,فرهنگ,Irna,https://www.irna.ir/news/84101751/%D9%85%D8%AC%D9%85%D9%88%D8%B9%D9%87-%D8%B4%D8%B9%D8%B1-%DA%AF%DB%8C%D9%88-%D9%86%D8%B5%DB%8C%D8%B1%DB%8C-%D9%85%D9%86%D8%AA%D8%B4%D8%B1-%D8%B4%D8%AF
2,668423b13d13baed495997c321bd0712db066d68,قیمت جهانی نفت امروز ۱۵ آذر ماه,به گزارش ایمنا به نقل از رویترز، در حالی که اوپک برای کاهش بیشتر تولید نفت در اوایل سال ۲۰۲۰ توافق کرد قیمت نفت امروز با کاهش مواجه بود. بر اساس این گزارش، قیمت هر بشکه نفت برنت امروز با ۲۱ سنت معادل ۰٫۳ درصد کاهش به ۶۳ دلار و ۱۸ سنت رسید. قیمت نفت وست تگزاس اینترمدییت آمریکا هم امروز با ۱۴ سنت معادل ۰٫۲ درصد کاهش به ۵۸ دلار و ۲۹ سنت رسید. قیمت نفت خام آمریکا روز پنجشنبه به بالاترین رقم از پایان ماه سپتامبر یعنی ۵۹ دلار و ۱۲ سنت رسید. سازمان کشورهای صادر کننده نفت (اوپک) و متحدانش از جمله روسیه موسوم [...],با وجود تصمیم اوپک برای کاهش بیشتر تولید نفت، قیمت هر بشکه نفت برنت امروز با ۲۱ سنت معادل ۰٫۳ درصد کاهش به ۶۳ دلار و ۱۸ سنت رسید.,Economy,اقتصادی,Imna,https://www.imna.ir/news/399763/%D9%82%DB%8C%D9%85%D8%AA-%D8%AC%D9%87%D8%A7%D9%86%DB%8C-%D9%86%D9%81%D8%AA-%D8%A7%D9%85%D8%B1%D9%88%D8%B2-%DB%B1%DB%B5-%D8%A2%D8%B0%D8%B1-%D9%85%D8%A7%D9%87
3,f15ec5a34191c464bdf79b115dc0d1767f3b906a,تکمیل پروژه گازرسانی به منطقه الموت قبل از فصل سرما,به گزارش شانا به نقل از پایگاه اطلاع رسانی وزارت کشور، مرتضی روزبه استاندار قزوین گفت: هم اکنون لوله‌های لازم برای اجرای طرح گاز رسانی به منطقه الموت خریداری شده و ظرف یک هفته آینده توسط پیمانکار به محل پروژه منتقل می‌شود. استاندار قزوین در ادامه افزود: مراحل اجرای طرح گاز رسانی به منطقه الموت با وجود صعب العبور بودن و اجرای ترانشه‌های ۹۰ متری در برخی نقاط، دارای روند خوبی است و در صورت تامین منابع مالی امیدواریم زمستان امسال مردم منطقه الموت از مزایای نعمت گاز طبیعی برخوردار شوند. وی تصریح کرد: برای اج [...],استاندار قزوین از تکمیل پروژه گاز رسانی به منطقه الموت قبل از فرا رسیدن فصل سرما خبر داد.,Oil-Energy,گاز,Shana,https://www.shana.ir/news/218907/%D8%AA%DA%A9%D9%85%DB%8C%D9%84-%D9%BE%D8%B1%D9%88%DA%98%D9%87-%DA%AF%D8%A7%D8%B2%D8%B1%D8%B3%D8%A7%D9%86%DB%8C-%D8%A8%D9%87-%D9%85%D9%86%D8%B7%D9%82%D9%87-%D8%A7%D9%84%D9%85%D9%88%D8%AA-%D9%82%D8%A8%D9%84-%D8%A7%D8%B2-%D9%81%D8%B5%D9%84-%D8%B3%D8%B1%D9%85%D8%A7
4,e8014f80

In [ ]:
# Display a sample of the dataset
# Use a smaller subset of the dataset for faster training
# Shuffle and select a smaller subset for training and validation
train = data["train"].shuffle(seed=42).select(range(1000))  # Use the first 1000 examples for training
validation = data["validation"].shuffle(seed=42).select(range(200))  # Use the first 200 examples for validation


indices = np.random.randint(0, len(train), 10).tolist()
df = pd.DataFrame(train[indices])
df = df[["id", "title", "article", "summary", "category", "categories", "network", "link"]]
df["article"] = df["article"].apply(lambda t: t.replace('[n]', ' ')[:512] + ' [...]')
df["category"] = df["category"].apply(lambda t: train.features["category"].int2str(t))
df["network"] = df["network"].apply(lambda t: train.features["network"].int2str(t))

# Display the sample dataframe
display.display(display.HTML(df.to_html(border=2, show_dimensions=True, justify='right', classes='xxx')))


,id,title,article,summary,category,categories,network,link
0,19bdd32376580c1ac323303f7f38f0d3dfae4d40,ورود شاخص کل بورس به کانال ۳۵۰ هزار واحد امروز ۲۷ آذر,به گزارش خبرنگار ایمنا، شاخص بورس تهران امروز چهارشنبه ۲۷ آذر ماه با افزایش سه هزار و ۴۵۲ واحدی عدد ۳۵۰ هزار و ۳۲۳ واحد را نمایش داد. شاخص هم وزن نیز به رقم ۱۱۰ هزار و ۹۰۷ واحد رسید. شاخص بازار اول به کانال ۲۵۰ هزار واحدی وارد شد و شاخص بازار دوم نیز عدد ۷۳۱ هزار واحد را نمایش داد. امروز آیفکس نیز چهار هزار و ۵۳۱ واحد را ثبت کرد. بر این اساس امروز با دست به دست شدن پنج میلیارد و ۶۲ میلیون سهم و اوراق بهادار طی ۵۷۱ هزار نوبت داد و ستد، ارزش معاملات به بیش از دو هزار و ۵۸۲ میلیارد تومان رسید. در معاملات ا [...],شاخص کل بورس تهران که از ابتدای روند معاملات امروز افزایشی شده بود پس از تجربه دو پالس منفی در طول فعالیت خود در نهایت با سه هزار و ۴۵۲ واحد افزایش به کار خود پایان داد.,Economy,اقتصادی,Imna,https://www.imna.ir/news/401775/%D9%88%D8%B1%D9%88%D8%AF-%D8%B4%D8%A7%D8%AE%D8%B5-%DA%A9%D9%84-%D8%A8%D9%88%D8%B1%D8%B3-%D8%A8%D9%87-%DA%A9%D8%A7%D9%86%D8%A7%D9%84-%DB%B3%DB%B5%DB%B0-%D9%87%D8%B2%D8%A7%D8%B1-%D9%88%D8%A7%D8%AD%D8%AF-%D8%A7%D9%85%D8%B1%D9%88%D8%B2-%DB%B2%DB%B7-%D8%A2%D8%B0%D8%B1
1,cebb3b2e10010167de444a2e0ceab33e1ca52dbc,مدرک تحصیلی دوم فرهنگیان در رتبه شغلی اعمال خواهد شد,به گزارش بازار به نقل از خانه ملت، نمایندگان در نشست علنی امروز (سه شنبه، ۱۱ آذرماه) مجلس شورای اسلامی در جریان بررسی گزارش کمیسیون برنامه و بودجه و محاسبات در مورد طرح یک فوریتی تنظیم برخی از مقررات مالی، اداری و پشتیبانی وزارت آموزش و پرورش با اصلاح تبصره ماده (۳) این قانون جهت تامین نظر شورای نگهبان موافقت کردند. در تبصره اصلاحی آمده است: وزارت آموزش و پرورش مکلف است در سال ۱۳۹۹ نسبت به اعمال مدرک تحصیلی دوم افرادی که فعالیت آموزشی در مدارس دارند، اقدام کند. بار مالی این تبصره در سقف اعتبار پیش‌بینی‌شد [...],نمایندگان مجلس شورای اسلامی مقرر کردند که مدرک تحصیلی دوم فرهنگیان و افراد شاغل در مدارس در رتبه شغلی آن‌ها اعمال شود.,Local,ایران,Tahlilbazaar,https://www.tahlilbazaar.com/news/57681/%D9%85%D8%AF%D8%B1%DA%A9-%D8%AA%D8%AD%D8%B5%DB%8C%D9%84%DB%8C-%D8%AF%D9%88%D9%85-%D9%81%D8%B1%D9%87%D9%86%DA%AF%DB%8C%D8%A7%D9%86-%D8%AF%D8%B1-%D8%B1%D8%AA%D8%A8%D9%87-%D8%B4%D8%BA%D9%84%DB%8C-%D8%A7%D8%B9%D9%85%D8%A7%D9%84-%D8%AE%D9%88%D8%A7%D9%87%D8%AF-%D8%B4%D8%AF
2,8658bab2342924ceb2d731abd72ef9df00655aa3,وزارت بهداشت: بیماران بهبود یافته پلاسما اهدا کنند,سیما سادات لاری، گفت: استفاده از پلاسمای خون بهبودیافتگان کرونا یکی از روش‌های درمانی برای درمان بیماران مبتلا به کووید۱۹ در بیمارستان‌ها است و تزریق آن در ابتدای بیماری می‌تواند در جلوگیری از بدحال شدن بیماران موثر باشد. وی افزود: میران مراجعه برای اهدای پلاسمای بهبودیافتگان از ابتدای اردیبهشت ماه تا امروز (۲۹ آبان‌ماه) در کل کشور ۱۳ هزار و ۷۰۰ نفر بوده است که از این میزان مراجعه تاکنون ۱۱ هزار و ۲۷۰ واحد پلاسما جمع‌آوری شده است. تاکنون تنها ۲ درصد از بهبودیافتگان از بیماری کووید۱۹ جهت اهدای پلاسمابه مرا [...],میزان نوشت: سخنگوی وزارت بهداشت با اشاره به نقش پلاسمادرمانی در بهبود مبتلایان کرونا از بهبودیافتگان این بیماری خواست تا با مراجعه به مراکز انتقال خون اقدام به اهدای پلاسما کنند,Society,اخبار اجتماعی+سلامت,Khabaronline,https://www.khabaronline.ir/news/1457404/%D9%88%D8%B2%D8%A7%D8%B1%D8%AA-%D8%A8%D9%87%D8%AF%D8%A7%D8%B4%D8%AA-%D8%A8%DB%8C%D9%85%D8%A7%D8%B1%D8%A7%D9%86-%D8%A8%D9%87%D8%A8%D9%88%D8%AF-%DB%8C%D8%A7%D9%81%D8%AA%D9%87-%D9%BE%D9%84%D8%A7%D8%B3%D9%85%D8%A7-%D8%A7%D9%87%D8%AF%D8%A7-%DA%A9%D9%86%D9%86%D8%AF
3,9e38c0dcca7ea894bfb53d2d6f826492393d4bd4,وزیر نفت: سی‌ان‌جی؛ سوخت ملی ماست,به گزارش خبرنگار شانا، بیژن زنگنه صبح امروز (یکشنبه، ۲۴ آذرماه) در آیین امضای تفاهم‌نامه تامین مالی طرح توسعه و تولید کارخانه‌ای و کارگاهی خودروهای دوگانه‌سوز بین شرکت ملی پخش فرآورده‌های نفتی و بانک ملت، با قدردانی از بانک ملت که مانند همیشه پیشتاز همکاری در کارهای صنعت نفت است و همچنین وزارت کشور، سازمان برنامه و بودجه و همه اشخاصی که با همت و اراده خود تلاش کردند مصوبه شورای اقتصاد این طرح ظرف دو روز دریافت و ابلاغ شود، اظهار کرد: این بی‌انصافی است اگر کاری تمام می‌شود یک نفر بخواهد به نام خود تمام کند؛ [...],وزیر نفت از گاز و سی‌ان‌جی به‌عنوان سوخت ملی نام بر

In [ ]:
# Initialize the tokenizer and model
model_name = "google/mt5-small"  # or "t5-small" for T5
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Preprocess the dataset
def preprocess_data(examples):
    inputs = examples["article"]
    targets = examples["summary"]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=150, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized_data = train.map(preprocess_data, batched=True)
validation_tokenized_data = validation.map(preprocess_data, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Install necessary libraries
!pip install -accelerate>=0.21.0


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -a


In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,  # Enable mixed precision training

)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_data,
    eval_dataset=validation_tokenized_data,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Fine-tune the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,nan


TrainOutput(global_step=250, training_loss=0.0, metrics={'train_runtime': 61.4056, 'train_samples_per_second': 16.285, 'train_steps_per_second': 4.071, 'total_flos': 528749690880000.0, 'train_loss': 0.0, 'epoch': 1.0})

In [ ]:
# Save the model
trainer.save_model("./fine-tuned-mt5")
tokenizer.save_pretrained("./fine-tuned-mt5")

('./fine-tuned-mt5/tokenizer_config.json',
 './fine-tuned-mt5/special_tokens_map.json',
 './fine-tuned-mt5/spiece.model',
 './fine-tuned-mt5/added_tokens.json')

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# Load the fine-tuned model and tokenizer
model_name = "/content/fine-tuned-mt5"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Function to generate a summary
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the model with an example article
example_article = """
به گزارش خبرنگار بازار، بیش از دو میلیون سهامدار بورس در مازندران وجود دارد که علاقه و انگیزه شان به تالار شیشه‌ای در ماه‌های اخیر درپی منفی و قرمز شدن این بازار روزبه روز کاسته شده است. بررسی و تحلیل آمارهای بورس مازندران نشان می‌دهد که هر هفته خیل فعالان این بازار در تالار شیشه‌ای استان کمتر می‌شود و کفه فروشندگان نسبت به خریداران آن هر روز سنگین‌تر می‌شود. کارشناسان و متولیان امر دلیل نابسامانی در بازار بورس را دخالت دولت و علت منفی بودن ماه‌های اخیر را دستوری بودن بیان می‌کنند. این درحالی است که ع
"""

# Generate the summary
summary = generate_summary(example_article)
print("Generated Summary:", summary)


Generated Summary: <extra_id_0> بورس مازندران
